In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from numpy.random import RandomState
from sklearn import metrics
from sklearn.utils import shuffle
import os
pd.set_option('display.max_colwidth', -1)

In [27]:
os.getcwd()

'/Users/yiweiluo/scientific-debates/cc_stance'

In [72]:
labeled_data = pd.read_csv('./tweet-stance-prediction/data/labeled_data_df')
#{'source':[],'type':[],'text':[],'stance':[]}
#unlabeled_data = {'source':[],'type':[],'text':[]}

In [73]:
labeled_data.shape

(157599, 5)

In [74]:
labeled_data.type.value_counts()

dallas_news_predicted    114686
dallas_news              40309 
mturk                    2040  
semeval_train_tweets     395   
semeval_tweets           169   
Name: type, dtype: int64

In [68]:
labeled_data.columns

Index(['Unnamed: 0', 'source', 'type', 'text', 'stance'], dtype='object')

In [69]:
labeled_data = labeled_data[['source','type','text','stance']]

# Dallas ELMO predicted labeled news

In [59]:
import json
lines = []
with open('data/all-predicted-sentences.jsonlist') as f:
    for line in f:
        lines.append(json.loads(line))

In [61]:
len(lines)

114686

In [60]:
lines[:3]

[{'article_id': 'climate_change1.0-6365',
  'sent_index': 0,
  'tokens': ['No',
   'Accord',
   ',',
   'and',
   'Little',
   'Optimism',
   ',',
   'at',
   'Bonn',
   'Talks',
   'on',
   'Global',
   'Warming'],
  'primary_tone': 'Pro'},
 {'article_id': 'climate_change1.0-6365',
  'sent_index': 1,
  'tokens': ['Delegates',
   'from',
   '143',
   'countries',
   'ended',
   'their',
   'preliminary',
   'talks',
   'on',
   'global',
   'warming',
   'here',
   'today',
   'with',
   'no',
   'agreement',
   'on',
   'key',
   'issues',
   'and',
   'growing',
   'concern',
   'about',
   'the',
   'prospect',
   'for',
   'success',
   'at',
   'the',
   'concluding',
   'meeting',
   ',',
   'to',
   'be',
   'held',
   'in',
   'Kyoto',
   ',',
   'Japan',
   ',',
   'in',
   'early',
   'December',
   '.'],
  'primary_tone': 'Pro'},
 {'article_id': 'climate_change1.0-6365',
  'sent_index': 2,
  'tokens': ['The',
   'United',
   'States',
   "'",
   'long-awaited',
   'proposal'

In [62]:
Counter([line['primary_tone'] for line in lines])

Counter({'Pro': 113777, 'Anti': 909})

In [64]:
labeled_data = {'source':[],'type':[],'text':[],'stance':[]}
labeled_data['source'] = [l['article_id'] + ' [SEP] ' + str(l['sent_index']) 
                          for l in lines]
labeled_data['type'] = ['dallas_news_predicted']*len(lines)
labeled_data['text'] = [' '.join(l['tokens'])[:-2]+'.' for l in lines]
labeled_data['stance'] = [tone2label[l['primary_tone'].lower()] for l in lines]
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-6365 [SEP] 0,dallas_news_predicted,"No Accord , and Little Optimism , at Bonn Talks on Global Warmi.",agree
1,climate_change1.0-6365 [SEP] 1,dallas_news_predicted,"Delegates from 143 countries ended their preliminary talks on global warming here today with no agreement on key issues and growing concern about the prospect for success at the concluding meeting , to be held in Kyoto , Japan , in early December.",agree
2,climate_change1.0-6365 [SEP] 2,dallas_news_predicted,The United States ' long-awaited proposal for cutting emissions of heat-trapping greenhouse gases has been scornfully attacked as too weak by delegates and environmentalists from around the world.,agree
3,climate_change1.0-6365 [SEP] 3,dallas_news_predicted,And there is no sign that American negotiators are about to yield.,agree
4,climate_change1.0-6365 [SEP] 4,dallas_news_predicted,"Still , the Americans themselves said there was more than a little cause for hope , and they have been lobbying their counterparts here , urging them to take a closer look at the American proposal.",agree
5,climate_change1.0-6365 [SEP] 5,dallas_news_predicted,"Meanwhile , several small-scale but higher-level meetings will be held between now and Dec. 1 , when the Kyoto conference begins.",agree
6,climate_change1.0-6365 [SEP] 6,dallas_news_predicted,"The goal of the conference is to revise the climate treaty negotiated in Rio in 1992 , which set voluntary limits on emissions.",agree
7,climate_change1.0-6365 [SEP] 7,dallas_news_predicted,"`` There are enough ideas out there to have an agreement , '' a senior State Department official said here , speaking on condition of anonymity.",agree
8,climate_change1.0-6365 [SEP] 8,dallas_news_predicted,`` We just do n't see all the kinds of coalescence that you need yet '' for that to happen.,agree
9,climate_change1.0-6365 [SEP] 9,dallas_news_predicted,"On the surface , the American proposal looks like the weakest on the table.",agree


In [70]:
labeled_data_df.shape

(114686, 4)

In [71]:
labeled_data = labeled_data.append(labeled_data_df, ignore_index = True)
labeled_data.to_csv('./tweet-stance-prediction/data/labeled_data_df')

# Dallas new labeled news

In [32]:
import json
lines = []
with open('data/all-annotated-sentences.jsonlist') as f:
    for line in f:
        lines.append(json.loads(line))

In [33]:
len(lines)

40790

In [34]:
lines[:5]

[{'article_id': 'climate_change1.0-12174',
  'sent_index': 0,
  'tokens': ['It',
   "'s",
   'about',
   'seeing',
   'the',
   'forest',
   'and',
   'the',
   'trees',
   'in',
   'Calif.',
   ';',
   'Scientists',
   'study',
   'global',
   'warming',
   "'s",
   'effect',
   'on',
   'redwoods',
   'and',
   'sequoias'],
  'primary_tone': 'Pro'},
 {'article_id': 'climate_change1.0-12174',
  'sent_index': 1,
  'tokens': ['Two',
   'of',
   'his',
   'colleagues',
   'dangled',
   'on',
   'ropes',
   '100',
   'feet',
   'above',
   'from',
   'the',
   'gnarled',
   'branches',
   'of',
   'a',
   'giant',
   'sequoia',
   'tree',
   '.'],
  'primary_tone': 'Pro'},
 {'article_id': 'climate_change1.0-12174',
  'sent_index': 2,
  'tokens': ['Steve',
   'Sillett',
   'could',
   'hardly',
   'contain',
   'his',
   'excitement',
   '.'],
  'primary_tone': 'Pro'},
 {'article_id': 'climate_change1.0-12174',
  'sent_index': 3,
  'tokens': ['``', 'This', 'site', 'is', 'just', 'juicy', '.

In [35]:
Counter([line['primary_tone'] for line in lines])

Counter({'Pro': 30917, 'Neutral': 3685, 'Anti': 6188})

In [40]:
tone2label = {'pro':'agree','anti':'disagree','neutral':'neutral'}

In [37]:
neutral_lines = [l for l in lines if l['primary_tone'] == 'Neutral']
len(neutral_lines)

3685

In [41]:
# Add all the neutrals to labeled_data_df
labeled_data = {'source':[],'type':[],'text':[],'stance':[]}
labeled_data['source'] = [l['article_id'] + ' [SEP] ' + str(l['sent_index']) 
                          for l in neutral_lines]
labeled_data['type'] = ['dallas_news']*len(neutral_lines)
labeled_data['text'] = [' '.join(l['tokens']) for l in neutral_lines]
labeled_data['stance'] = [tone2label[l['primary_tone'].lower()] for l in neutral_lines]
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-9251 [SEP] 0,dallas_news,From Nuclear Freeze to Global Warming - and Back,neutral
1,climate_change1.0-9251 [SEP] 1,dallas_news,"In June 1982 , up to a million demonstrators gathered in Central Park calling for a nuclear freeze .",neutral
2,climate_change1.0-9251 [SEP] 2,dallas_news,"They were protesting the Reagan-era nuclear arms buildup and other developments they saw , not unreasonably , as a threat to civilization and to life on Earth , including talk by some Reagan aides about fighting and winning a nuclear war .",neutral
3,climate_change1.0-9251 [SEP] 3,dallas_news,"Last Sunday - a generation later - hundreds of thousands of demonstrators took part in the People 's Climate March in Manhattan to protest international inaction on global warming , which they see , not unreasonably , as a threat to civilization and to life on Earth.The front page of The New York Times on Monday brought the two events together , in an unintended though instructive way .",neutral
4,climate_change1.0-9251 [SEP] 4,dallas_news,The photographs were of the climate marchers .,neutral
5,climate_change1.0-9251 [SEP] 5,dallas_news,But the lead story was about the current aggressive efforts to rebuild the nation 's nuclear arsenal .,neutral
6,climate_change1.0-9251 [SEP] 6,dallas_news,"The fact that , 32 years after the Central Park gathering , progress on significant arms reductions is going into reverse is something the climate marchers should note .",neutral
7,climate_change1.0-9251 [SEP] 7,dallas_news,"Because , intentionally or not , the climate change movement mirrors the nuclear freeze in many ways .",neutral
8,climate_change1.0-9251 [SEP] 8,dallas_news,Both are rooted in the idea of a global planetary emergency .,neutral
9,climate_change1.0-9251 [SEP] 9,dallas_news,Both grew along with scientific evidence that challenges the status quo in politics and industry .,neutral


In [42]:
labeled_data_df.shape

(3685, 4)

In [47]:
labeled_data.shape

(39228, 5)

In [58]:
39228+3685

42913

In [53]:
labeled_data = labeled_data.append(labeled_data_df, ignore_index = True)
labeled_data.to_csv('./tweet-stance-prediction/data/labeled_data_df')

# Dallas labeled news

In [15]:
import json
lines = []
with open('data/all.pro-anti.sentences.jsonlist') as f:
    for line in f:
        lines.append(json.loads(line))

In [16]:
len(lines)

37105

In [22]:
lines[:5]

[{'article_id': 'climate_change1.0-12174',
  'sent_index': 0,
  'text': "It's about seeing the forest and the trees in Calif.; Scientists study global warming's effect on redwoods and sequoias",
  'tokens': ['It',
   "'s",
   'about',
   'seeing',
   'the',
   'forest',
   'and',
   'the',
   'trees',
   'in',
   'Calif.',
   ';',
   'Scientists',
   'study',
   'global',
   'warming',
   "'s",
   'effect',
   'on',
   'redwoods',
   'and',
   'sequoias'],
  'primary_tone': 'Pro'},
 {'article_id': 'climate_change1.0-12174',
  'sent_index': 1,
  'text': 'Two of his colleagues dangled on ropes 100 feet above from the gnarled branches of a giant sequoia tree.',
  'tokens': ['Two',
   'of',
   'his',
   'colleagues',
   'dangled',
   'on',
   'ropes',
   '100',
   'feet',
   'above',
   'from',
   'the',
   'gnarled',
   'branches',
   'of',
   'a',
   'giant',
   'sequoia',
   'tree',
   '.'],
  'primary_tone': 'Pro'},
 {'article_id': 'climate_change1.0-12174',
  'sent_index': 2,
  'text'

In [38]:
Counter([l['primary_tone'].lower() for l in lines])

Counter({'pro': 30917, 'anti': 6188})

In [39]:
tone2label = {'pro':'agree','anti':'disagree'}

In [58]:

labeled_data['source'] = [l['article_id'] + ' [SEP] ' + str(l['sent_index']) for l in lines]
labeled_data['type'] = ['dallas_news']*len(lines)
labeled_data['text'] = [l['text'] for l in lines]
labeled_data['stance'] = [tone2label[l['primary_tone'].lower()] for l in lines]
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
5,climate_change1.0-12174 [SEP] 5,dallas_news,"""There are amazing lichens up there.",agree
6,climate_change1.0-12174 [SEP] 6,dallas_news,There are aphids being hunted by ladybugs.,agree
7,climate_change1.0-12174 [SEP] 7,dallas_news,I found a sizable dogwood tree growing from a ...,agree
8,climate_change1.0-12174 [SEP] 8,dallas_news,"The foggy, lost worlds atop the world's bigges...",agree
9,climate_change1.0-12174 [SEP] 9,dallas_news,Concerned that rising temperatures across Cali...,agree


# Dallas unlabeled news

In [9]:
os.getcwd()

'/Users/yiweiluo/scientific-debates/cc_stance'

In [25]:
all_sents = []
with open('data/all.sentences.jsonlist') as f:
    for line in f:
        all_sents.append(json.loads(line))

In [11]:
len(all_sents)

126100

In [21]:
len([line['primary_tone'] for line in all_sents if 'primary_tone' in line])

0

In [26]:
all_sents[:10]

[{'article_id': 'climate_change1.0-6365',
  'sent_index': 0,
  'text': 'No Accord, and Little Optimism, at Bonn Talks on Global Warming',
  'tokens': ['No',
   'Accord',
   ',',
   'and',
   'Little',
   'Optimism',
   ',',
   'at',
   'Bonn',
   'Talks',
   'on',
   'Global',
   'Warming']},
 {'article_id': 'climate_change1.0-6365',
  'sent_index': 1,
  'text': 'Delegates from 143 countries ended their preliminary talks on global warming here today with no agreement on key issues and growing concern about the prospect for success at the concluding meeting, to be held in Kyoto, Japan, in early December.',
  'tokens': ['Delegates',
   'from',
   '143',
   'countries',
   'ended',
   'their',
   'preliminary',
   'talks',
   'on',
   'global',
   'warming',
   'here',
   'today',
   'with',
   'no',
   'agreement',
   'on',
   'key',
   'issues',
   'and',
   'growing',
   'concern',
   'about',
   'the',
   'prospect',
   'for',
   'success',
   'at',
   'the',
   'concluding',
   'meet

In [20]:
'climate_change1.0-6365 [SEP] 0' in labeled_data_df['source']

False

In [84]:
#all_sents[1]

In [178]:
unlabeled_data['source'] = [l['article_id'] + ' [SEP] ' + str(l['sent_index']) 
                               for l in all_sents]
unlabeled_data['type'] = ['dallas_news']*len(all_sents)
unlabeled_data['text'] = [l['text'] for l in all_sents]
unlabeled_data_df = pd.DataFrame(unlabeled_data)
unlabeled_data_df.head(10)

,source,type,text
0,climate_change1.0-6365 [SEP] 0,dallas_news,"No Accord, and Little Optimism, at Bonn Talks ..."
1,climate_change1.0-6365 [SEP] 1,dallas_news,Delegates from 143 countries ended their preli...
2,climate_change1.0-6365 [SEP] 2,dallas_news,The United States' long-awaited proposal for c...
3,climate_change1.0-6365 [SEP] 3,dallas_news,And there is no sign that American negotiators...
4,climate_change1.0-6365 [SEP] 4,dallas_news,"Still, the Americans themselves said there was..."
5,climate_change1.0-6365 [SEP] 5,dallas_news,"Meanwhile, several small-scale but higher-leve..."
6,climate_change1.0-6365 [SEP] 6,dallas_news,The goal of the conference is to revise the cl...
7,climate_change1.0-6365 [SEP] 7,dallas_news,"""There are enough ideas out there to have an a..."
8,climate_change1.0-6365 [SEP] 8,dallas_news,"""We just don't see all the kinds of coalescenc..."
9,climate_change1.0-6365 [SEP] 9,dallas_news,"On the surface, the American proposal looks li..."


# SemEval tweets

In [4]:
targets = []
with open(os.getcwd()+'/data/semeval2016-task6-testdata-gold/SemEval2016-Task6-subtaskB-testdata\
-gold.txt') as f:
    semeval_b = f.readlines()
    for line in semeval_b[1:]:
        id_,target,tweet,stance = line.strip().split('\t')
        targets.append(target)

In [5]:
from collections import Counter
Counter(targets)

Counter({'Donald Trump': 707})

In [6]:
semeval_tweets = []
with open(os.getcwd()+'/data/semeval2016-task6-testdata-gold/SemEval2016-Task6-subtaskA-testdata\
-gold.txt') as f:
    semeval_a = f.readlines()
    for line in semeval_a[1:]:
        id_,target,tweet,stance = line.strip().split('\t')
        if target == 'Climate Change is a Real Concern':
            semeval_tweets.append(line.strip().split('\t'))

In [7]:
len(semeval_tweets)

169

In [8]:
from collections import Counter
Counter([x[-1] for x in semeval_tweets])

Counter({'NONE': 35, 'FAVOR': 123, 'AGAINST': 11})

In [9]:
tweetstance2label = {'NONE':'neutral','FAVOR':'agree','AGAINST':'disagree'}

In [11]:
labeled_data['source'].extend([l[0] for l in semeval_tweets])
labeled_data['type'].extend(['semeval_tweets']*len(semeval_tweets))
labeled_data['text'].extend([l[2] for l in semeval_tweets])
labeled_data['stance'].extend([tweetstance2label[l[3]] for l in semeval_tweets])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,10221,semeval_tweets,Closed door session begins. More after they de...,neutral
1,10222,semeval_tweets,What is the #energiewende agenda? Why are they...,agree
2,10223,semeval_tweets,@climasphere Stocker: Fish catch potential cou...,agree
3,10224,semeval_tweets,"...a longer memory, and a sterner sense of jus...",neutral
4,10225,semeval_tweets,"Boni ""Future Sea level rise will not be unifor...",agree
5,10226,semeval_tweets,@climasphere Bony: Extreme weather events (hea...,agree
6,10227,semeval_tweets,Camille Parmesan: 2/3 of species have changed ...,agree
7,10228,semeval_tweets,5 meta-analyses show 1/2 of species have chang...,agree
8,10229,semeval_tweets,It literarly just took me 10 minutes to get in...,neutral
9,10230,semeval_tweets,Grubb: state of energy: shift US & EU from net...,agree


In [12]:
semeval_tweets

[['10221',
  'Climate Change is a Real Concern',
  'Closed door session begins. More after they decide on the entities.  #GCF #SemST',
  'NONE'],
 ['10222',
  'Climate Change is a Real Concern',
  'What is the #energiewende agenda? Why are they not in the "de-carbonizer" camp?.  How long will you tolerate the lies?  #SemST',
  'FAVOR'],
 ['10223',
  'Climate Change is a Real Concern',
  '@climasphere Stocker: Fish catch potential could drop by as much as 50% in some areas due to #oceanacidification. #CFCC15 #SemST',
  'FAVOR'],
 ['10224',
  'Climate Change is a Real Concern',
  '...a longer memory, and a sterner sense of justice than we do. -W.Berry #SemST',
  'NONE'],
 ['10225',
  'Climate Change is a Real Concern',
  'Boni "Future Sea level rise will not be uniform & different for coasts" #CCFC15 #Journey2015 #SemST',
  'FAVOR'],
 ['10226',
  'Climate Change is a Real Concern',
  '@climasphere Bony: Extreme weather events (heat waves, drought, flooding, windstorms) made more frequent

In [43]:
labeled_data_df.tail(10)

,source,type,text,stance
37264,10380,semeval_tweets,@4EvaMartino Everyone should learn this recip...,agree
37265,10381,semeval_tweets,Rec high temp for this date in Charlotte is 10...,agree
37266,10382,semeval_tweets,"Lot's of time, class over! @nationaljournal #S...",neutral
37267,10383,semeval_tweets,@TheLibertyEagle Oh for the love of Pete! I th...,neutral
37268,10384,semeval_tweets,If complaining about irrelevant problems were ...,neutral
37269,10385,semeval_tweets,"@LiberalAus - ""All"" you folks #owe me an #offi...",neutral
37270,10386,semeval_tweets,"Maybe we should start ""labelling every #produc...",agree
37271,10387,semeval_tweets,If we touch #Antarctica for one more #reason o...,agree
37272,10388,semeval_tweets,Wind power helped stabilize Texas' power grid ...,agree
37273,10389,semeval_tweets,"It's so cold and windy here in Sydney, I just ...",agree


In [13]:
stance_test = pd.read_csv(open(os.getcwd()+'/data/StanceDataset/test.csv','rU'),
                          encoding='utf-8', engine='c')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


In [14]:
stance_test = stance_test[stance_test['Target'] == 'Climate Change is a Real Concern']

In [15]:
stance_test.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
220,Closed door session begins. More after they de...,Climate Change is a Real Concern,NONE,3. The tweet is not explicitly expressing opi...,other
221,What is the #energiewende agenda? Why are they...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
222,@climasphere Stocker: Fish catch potential cou...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,neg
223,"...a longer memory, and a sterner sense of jus...",Climate Change is a Real Concern,NONE,2. The tweet does NOT expresses opinion about ...,other
224,"Boni ""Future Sea level rise will not be unifor...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other


In [16]:
labeled_data['source'].extend(list(stance_test.index))
labeled_data['type'].extend(['semeval_test_tweets']*len(stance_test))
labeled_data['text'].extend([stance_test.loc[l]['Tweet'] for l in stance_test.index])
labeled_data['stance'].extend([tweetstance2label[stance_test.loc[l]['Stance']] 
                               for l in stance_test.index])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,10221,semeval_tweets,Closed door session begins. More after they de...,neutral
1,10222,semeval_tweets,What is the #energiewende agenda? Why are they...,agree
2,10223,semeval_tweets,@climasphere Stocker: Fish catch potential cou...,agree
3,10224,semeval_tweets,"...a longer memory, and a sterner sense of jus...",neutral
4,10225,semeval_tweets,"Boni ""Future Sea level rise will not be unifor...",agree
5,10226,semeval_tweets,@climasphere Bony: Extreme weather events (hea...,agree
6,10227,semeval_tweets,Camille Parmesan: 2/3 of species have changed ...,agree
7,10228,semeval_tweets,5 meta-analyses show 1/2 of species have chang...,agree
8,10229,semeval_tweets,It literarly just took me 10 minutes to get in...,neutral
9,10230,semeval_tweets,Grubb: state of energy: shift US & EU from net...,agree


In [61]:
labeled_data_df.tail(10)

,source,type,text,stance
37433,379,semeval_test_tweets,@4EvaMartino Everyone should learn this recip...,agree
37434,380,semeval_test_tweets,Rec high temp for this date in Charlotte is 10...,agree
37435,381,semeval_test_tweets,"Lot's of time, class over! @nationaljournal #S...",neutral
37436,382,semeval_test_tweets,@TheLibertyEagle Oh for the love of Pete! I th...,neutral
37437,383,semeval_test_tweets,If complaining about irrelevant problems were ...,neutral
37438,384,semeval_test_tweets,"@LiberalAus - ""All"" you folks #owe me an #offi...",neutral
37439,385,semeval_test_tweets,"Maybe we should start ""labelling every #produc...",agree
37440,386,semeval_test_tweets,If we touch #Antarctica for one more #reason o...,agree
37441,387,semeval_test_tweets,Wind power helped stabilize Texas' power grid ...,agree
37442,388,semeval_test_tweets,"It's so cold and windy here in Sydney, I just ...",agree


In [18]:
stance_train = pd.read_csv(open(os.getcwd()+'/data/StanceDataset/train.csv','rU'),
                          encoding='utf-8', engine='c')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.


In [19]:
stance_train = stance_train[stance_train['Target'] == 'Climate Change is a Real Concern']

In [20]:
stance_train.head()

,Tweet,Target,Stance,Opinion Towards,Sentiment
613,"We cant deny it, its really happening. #SemST",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,other
614,RT @cderworiz: Timelines are short. Strategy m...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
615,SO EXCITING! Meaningful climate change action ...,Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
616,"Delivering good jobs for Albertans, maintainin...",Climate Change is a Real Concern,FAVOR,1. The tweet explicitly expresses opinion abo...,pos
617,@davidswann says he wants carbon fund to be sp...,Climate Change is a Real Concern,FAVOR,3. The tweet is not explicitly expressing opi...,other


In [24]:
labeled_data['source'].extend(list(stance_train.index))
labeled_data['type'].extend(['semeval_train_tweets']*len(stance_train))
labeled_data['text'].extend([stance_train.loc[l]['Tweet'] for l in stance_train.index])
labeled_data['stance'].extend([tweetstance2label[stance_train.loc[l]['Stance']] 
                               for l in stance_train.index])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,10221,semeval_tweets,Closed door session begins. More after they decide on the entities. #GCF #SemST,neutral
1,10222,semeval_tweets,"What is the #energiewende agenda? Why are they not in the ""de-carbonizer"" camp?. How long will you tolerate the lies? #SemST",agree
2,10223,semeval_tweets,@climasphere Stocker: Fish catch potential could drop by as much as 50% in some areas due to #oceanacidification. #CFCC15 #SemST,agree
3,10224,semeval_tweets,"...a longer memory, and a sterner sense of justice than we do. -W.Berry #SemST",neutral
4,10225,semeval_tweets,"Boni ""Future Sea level rise will not be uniform & different for coasts"" #CCFC15 #Journey2015 #SemST",agree
5,10226,semeval_tweets,"@climasphere Bony: Extreme weather events (heat waves, drought, flooding, windstorms) made more frequent/intense by #CFCC15 #SemST",agree
6,10227,semeval_tweets,Camille Parmesan: 2/3 of species have changed #habitat because of #H12 #CFCC15 #SemST,agree
7,10228,semeval_tweets,"5 meta-analyses show 1/2 of species have changed where they live, 2/3 have changed when they live. Camille Parmesan #CFCC15 #SemST",agree
8,10229,semeval_tweets,It literarly just took me 10 minutes to get in a bus because an old lady just had to go in front of everyone... #SemST,neutral
9,10230,semeval_tweets,"Grubb: state of energy: shift US & EU from net-producer to net-importer of emissions"" #CCFC15 #journey2015 #SemST",agree


In [26]:
labeled_data_df[labeled_data_df.stance=='disagree']

,source,type,text,stance
99,10320,semeval_tweets,ONE Volcano emits more pollution than man has in our HISTORY! #SemST,disagree
109,10330,semeval_tweets,@UN ice cracking in the summer?? SO ALARMING. #climatechangehoax #SemST,disagree
120,10341,semeval_tweets,"#ClimateChangeDefinitions ""settled science"" = flawed computer models with jiggered data to force a predefined conclusion. #SemST",disagree
127,10348,semeval_tweets,@Myrmecos Still a larval theory. Still poorly modelled. Still unquantified. Still multi-factor. Still flummoxed by albedo. #SemST,disagree
136,10357,semeval_tweets,Climate deniers is a term used to silence those pointing out the hypocrisy in the fanatical zeal on #climatetruth #SemST,disagree
145,10366,semeval_tweets,"@wattsupwiththat yes ironic that the alarmists are the actual ""deniers #SemST",disagree
146,10367,semeval_tweets,@JustinTrudeau Hey Justin I will give you 50 cents if you stop talking about climate 'Change' #Ottawa #davidsuzuki #cbc #SemST,disagree
147,10368,semeval_tweets,If @EPA & @POTUS don't have to follow SCOTUS why should anyone else have to #ClimateFraud #SpecialReport #WakeUpAmerica #SemST,disagree
149,10370,semeval_tweets,"Govt announcement on zero carbon homes ""short-sighted, unnecessary, retrograde and damaging"" says @UKGBC #energyefficiency #SemST",disagree
151,10372,semeval_tweets,"We are not ""killing the Earth"". The Earth has been through worse and will be fine after all humans suffocate, drown or starve #SemST",disagree


# Dallas and Yiwei manually annotated news--not yet added

In [28]:
import json
dallas_labels = []
with open('subsample_annotated_dbc.jsonlist') as f:
    for line in f:
        dallas_labels.append(json.loads(line))

In [29]:
len(dallas_labels)

300

In [34]:
dallas_labels[10:20]

[{'comp_clause': "that measures such as conservation wo n't cost much and will buy insurance against global warming",
  'full': "Some argue that measures such as conservation won't cost much and will buy insurance against global warming.",
  'key': 106901,
  'label': 0},
 {'comp_clause': 'I became somewhat inspired',
  'full': '"I became somewhat inspired," he said.',
  'key': 143590,
  'label': 3},
 {'comp_clause': 'if he ’s joking',
  'full': '“I would argue, as would many other beeologists,” he says, “that most bees are arguably cuter than most kids.” It’s hard to tell if he’s joking.',
  'key': 61472,
  'label': 3},
 {'comp_clause': 'the Earth habitable',
  'full': 'Gases such as carbon dioxide trap the energy that pours down on the Earth from the sun, making the Earth habitable.',
  'key': 276801,
  'label': 2},
 {'comp_clause': 'Rescue workers tried to \xa0 expand the sweater idea to pelicans \xa0 during the Gulf spill in 2010',
  'full': 'Rescue workers tried to\xa0expand the sw

In [31]:
yiwei_labels = pd.read_csv('subsample_annotated_yiwei.csv',header=None)

In [32]:
yiwei_labels.head()

,0,1,2,3
0,19565,"Reading ""an early draft"" of a federal government report has helped David Ignatius see that the global warming debate is just like the tobacco wars of the past four decades [""Naysay Now, Pay Later,"" op-ed, Aug. 18].",that the global warming debate is just like the tobacco wars of the past four decades,1
1,92670,"We haven’t seen that in the observations,” Myles Allen, professor of geosystem science at the University of Oxford and one of the study’s authors, told the Times of London, as reported in The Independent.","that in the observations , ” Myles Allen , professor of geosystem science at the University of Oxford and one of the study ’s authors , told the Times of London , as reported in The Independent",2
2,152040,"Describing it as a rare hybrid “superstorm,” forecasters say Sandy was created by an Arctic jet stream wrapping itself around a tropical storm.",Sandy was created by an Arctic jet stream wrapping itself around a tropical storm,2
3,191315,JEFFERSON CITY o Ameren Missouri says it must decide quickly on how to comply with proposed Environmental Protection Agency rules limiting carbon pollution.,it must decide quickly on how to comply with proposed Environmental Protection Agency rules limiting carbon pollution,2
4,250921,"There was this exposé in the Times where you first were talking to Andrew Revkin and explaining what was happening under the Bush administration, and even before that, the suppression of your work when you testified before Congress to, what, Senator Al Gore at the time.","what was happening under the Bush administration ,",3


In [75]:
anti_test_data = []
pro_test_data = []
with open(os.getcwd()+'/test_data/anti-cc/test_data.txt') as f:
    for line in f:
        anti_test_data.append(line.split('\t'))
with open(os.getcwd()+'/test_data/pro-cc/test_data.txt') as f:
    for line in f:
        pro_test_data.append(line.split('\t'))

In [76]:
len(anti_test_data)

92804

In [77]:
anti_test_data[0]

['We ’re also doing some avian and marine mammal monitoring',
 '',
 'sent_no_29 of https://www.csmonitor.com/Environment/2010/1012/Will-Google-wind-power-project-harm-wildlife-Depends-on-location.',
 '',
 '“We’re also doing some avian and marine mammal monitoring,” says Rhonda Jackson, a spokesman for Fishermen’s Energy.\n']

In [78]:
len(pro_test_data)

581383

In [81]:
Counter([len(d) for d in pro_test_data])

Counter({5: 286327,
         1: 290241,
         17: 2316,
         6: 113,
         7: 59,
         9: 34,
         29: 2,
         14: 27,
         11: 33,
         8: 12,
         15: 1,
         53: 353,
         273: 56,
         292: 1,
         111: 91,
         106: 464,
         138: 459,
         271: 165,
         103: 209,
         145: 1,
         107: 12,
         60: 5,
         75: 11,
         54: 7,
         68: 6,
         132: 2,
         94: 2,
         128: 5,
         348: 17,
         95: 1,
         57: 1,
         283: 1,
         71: 11,
         354: 13,
         360: 1,
         380: 2,
         401: 1,
         74: 1,
         129: 2,
         69: 17,
         369: 2,
         414: 1,
         70: 4,
         77: 2,
         318: 4,
         117: 14,
         91: 7,
         139: 5,
         121: 8,
         130: 3,
         358: 5,
         362: 2,
         287: 1,
         169: 7,
         186: 4,
         203: 4,
         277: 6,
         382: 5,
      

# Yiwei unlabeled news

In [98]:
all_url_df = pd.read_pickle('/Users/yiweiluo/Dropbox/research/\
QP2/code/Fox_and_friends/all_urls_meta_and_fulltext_df_2020.pkl')

In [99]:
cc_all_url_df = all_url_df[all_url_df['topic'] == 'cc'][all_url_df['bad NYT'] == False]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [100]:
cc_all_url_df.head()

,bad NYT,date,domain,fulltext,stance,title,topic,url
1600,False,None,:///,If you can't sleep as much as you need durin...,anti,None,cc,foxnews.com/lifestyle/climate-change-might-be-...
1601,False,None,:///,Paper says draft document by scientists direct...,anti,None,cc,foxnews.com/opinion/climate-policy-get-ready-f...
1602,False,None,:///,The public's concern about global warming has ...,anti,None,cc,foxnews.com/opinion/physicist-dont-fall-for-th...
1603,False,None,:///,The federal report by dozens of U.S. governmen...,anti,None,cc,foxnews.com/politics/junk-science-studies-behi...
1604,False,None,:///,File photo - A statue of the Virgin Mary is ...,anti,None,cc,foxnews.com/science/10-years-after-hurricane-k...


In [102]:
cc_all_url_df.shape

(17601, 8)

In [181]:
from nltk.tokenize import sent_tokenize
for ix,row in cc_all_url_df.iterrows():
    source = row['url']
    type_ = 'yiwei_news'
    if row['fulltext'] is not None:
        text = sent_tokenize(row['fulltext'])
        unlabeled_data['text'].extend(text)
        unlabeled_data['type'].extend([type_]*len(text))
        unlabeled_data['source'].extend([source]*len(text))

In [192]:
unlabeled_data_df = pd.DataFrame(unlabeled_data)
unlabeled_data_df.head(10)

,source,type,text
0,climate_change1.0-6365 [SEP] 0,dallas_news,"No Accord, and Little Optimism, at Bonn Talks ..."
1,climate_change1.0-6365 [SEP] 1,dallas_news,Delegates from 143 countries ended their preli...
2,climate_change1.0-6365 [SEP] 2,dallas_news,The United States' long-awaited proposal for c...
3,climate_change1.0-6365 [SEP] 3,dallas_news,And there is no sign that American negotiators...
4,climate_change1.0-6365 [SEP] 4,dallas_news,"Still, the Americans themselves said there was..."
5,climate_change1.0-6365 [SEP] 5,dallas_news,"Meanwhile, several small-scale but higher-leve..."
6,climate_change1.0-6365 [SEP] 6,dallas_news,The goal of the conference is to revise the cl...
7,climate_change1.0-6365 [SEP] 7,dallas_news,"""There are enough ideas out there to have an a..."
8,climate_change1.0-6365 [SEP] 8,dallas_news,"""We just don't see all the kinds of coalescenc..."
9,climate_change1.0-6365 [SEP] 9,dallas_news,"On the surface, the American proposal looks li..."


In [193]:
unlabeled_data_df.tail(10)

,source,type,text
1158101,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"and ""do you have a signed ration card from you..."
1158102,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,You can stand by but there's no telling when a...
1158103,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Comrade.
1158104,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Now, a really good high-speed rail system woul..."
1158105,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Let's say something that can travel at 500 mil...
1158106,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that can go to every major city, and..."
1158107,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that doesn't need new rights of way,..."
1158108,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Something where the rights of way are as free ...
1158109,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,But hoping for something like that is just cra...
1158110,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,https://pjmedia.com/trending/aoc-and-the-magic...


# Clean, dedup, filter labeled and unlabeled data

In [110]:
# Functions for importing & cleaning labeled items
def lower(s):
    return s.lower()

def imports(filename):
    imp = pd.read_pickle(filename)
    imp = imp.drop_duplicates()
    imp['sent_clean'] = imp['sentence'].str.replace('http\S+|www.\S+|pic.twitter.com\S+', '', case=False)
    #imp['sent_clean'] = imp['sentence'].replace('[^A-Za-z0-9 ]+','',regex=True)
    imp['sent_clean'] = imp['sent_clean'].apply(lower)
    return imp

def clean(x):
    return x.replace('http\S+|www.\S+|pic.twitter.com\S+', '')

In [194]:
unlabeled_data_df = unlabeled_data_df.dropna()
unlabeled_data_df['text'] = unlabeled_data_df['text'].apply(clean)
unlabeled_data_df.drop_duplicates(subset= 'text',inplace=True)
unlabeled_data_df = unlabeled_data_df.reset_index()

In [195]:
labeled_data_df = labeled_data_df.dropna()
labeled_data_df['text'] = labeled_data_df['text'].apply(clean)
labeled_data_df.drop_duplicates(subset= 'text',inplace=True)
labeled_data_df = labeled_data_df.reset_index()

In [196]:
labeled_data_df = labeled_data_df.drop(['index'],axis=1)

In [197]:
unlabeled_data_df.shape

(866654, 4)

In [198]:
labeled_data_df.shape

(39228, 4)

In [ ]:
# summary of what's included

In [199]:
unlabeled_data_df.tail(10)

,index,source,type,text
866644,1158101,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"and ""do you have a signed ration card from you..."
866645,1158102,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,You can stand by but there's no telling when a...
866646,1158103,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Comrade.
866647,1158104,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Now, a really good high-speed rail system woul..."
866648,1158105,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Let's say something that can travel at 500 mil...
866649,1158106,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that can go to every major city, and..."
866650,1158107,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,"Something that doesn't need new rights of way,..."
866651,1158108,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,Something where the rights of way are as free ...
866652,1158109,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,But hoping for something like that is just cra...
866653,1158110,https://pjmedia.com/trending/aoc-and-the-magic...,yiwei_news,https://pjmedia.com/trending/aoc-and-the-magic...


In [1]:
unlabeled_data_df['type'].value_counts() 

NameError: name 'unlabeled_data_df' is not defined

In [ ]:
labeled_data_df

In [173]:
labeled_data_df.to_csv('labeled_data_df')

In [202]:
labeled_data_df['stance'].value_counts()

agree       31629
disagree     6534
neutral      1065
Name: stance, dtype: int64

In [203]:
labeled_data_df['type'].value_counts()

dallas_news             36624
mturk                    2040
semeval_train_tweets      395
semeval_tweets            169
Name: type, dtype: int64

In [3]:
labeled_data_df = pd.read_csv(os.getcwd()+'/tweet-stance-prediction/data/labeled_data_df')

In [4]:
labeled_data_df.shape

(39228, 5)

In [5]:
labeled_data_df.type.value_counts()

dallas_news             36624
mturk                   2040 
semeval_train_tweets    395  
semeval_tweets          169  
Name: type, dtype: int64

In [6]:
labeled_data_df.loc[labeled_data_df.type=='dallas_news'].stance.value_counts()

agree       30517
disagree    6107 
Name: stance, dtype: int64

In [39]:
mturk_data = labeled_data_df[labeled_data_df.type=='mturk']

In [40]:
mturk_data.shape

(2040, 5)

In [41]:
mturk_data.stance.value_counts()

neutral     862
agree       777
disagree    401
Name: stance, dtype: int64

In [158]:
dallas_sentence_scores = pd.read_csv('/Users/yiweiluo/scientific-debates/\
cc_stance/MTurk_results/sent_scores_df.tsv',delimiter='\t')
dallas_sentence_scores['max_prob_label'] = dallas_sentence_scores[['disagree',
                                                                  'neutral',
                                                                  'agree']].idxmax(axis=1)

In [161]:
labeled_data['source'].extend(['round{}_batch{}_sentid{}'.format(row['round'],
                                                              row['batch'],
                                                              row['sent_id'])
                              for ix,row in dallas_sentence_scores.iterrows()])
labeled_data['type'].extend(['mturk']*len(dallas_sentence_scores))
labeled_data['text'].extend([row['sentence'] for ix,row in 
                             dallas_sentence_scores.iterrows()])
labeled_data['stance'].extend([row['max_prob_label'] for ix,row in 
                             dallas_sentence_scores.iterrows()])
labeled_data_df = pd.DataFrame(labeled_data)
labeled_data_df.head(10)

,source,type,text,stance
0,climate_change1.0-12174 [SEP] 0,dallas_news,It's about seeing the forest and the trees in ...,agree
1,climate_change1.0-12174 [SEP] 1,dallas_news,Two of his colleagues dangled on ropes 100 fee...,agree
2,climate_change1.0-12174 [SEP] 2,dallas_news,Steve Sillett could hardly contain his excitem...,agree
3,climate_change1.0-12174 [SEP] 3,dallas_news,"""This site is just juicy.",agree
4,climate_change1.0-12174 [SEP] 4,dallas_news,"It's loaded,"" said Sillett, gazing up at the a...",agree
5,climate_change1.0-12174 [SEP] 5,dallas_news,"""There are amazing lichens up there.",agree
6,climate_change1.0-12174 [SEP] 6,dallas_news,There are aphids being hunted by ladybugs.,agree
7,climate_change1.0-12174 [SEP] 7,dallas_news,I found a sizable dogwood tree growing from a ...,agree
8,climate_change1.0-12174 [SEP] 8,dallas_news,"The foggy, lost worlds atop the world's bigges...",agree
9,climate_change1.0-12174 [SEP] 9,dallas_news,Concerned that rising temperatures across Cali...,agree


In [159]:
#dallas_sentence_scores.to_pickle('dallas_sent_scores.pkl')
dallas_sentence_scores.head()

,Unnamed: 0,round,batch,sent_id,disagree,neutral,agree,sentence,max_prob_label
0,0,1,0,t0,0.004241,0.260963,0.734797,Warmer-than-normal sea surface temperatures ar...,agree
1,1,1,0,t1,0.001548,0.996214,0.002239,We will continue to rely in part on fossil fue...,neutral
2,2,1,0,t10,0.001440,0.996503,0.002057,The actual rise in sea levels measured only 1....,neutral
3,3,1,0,t11,0.996815,0.001588,0.001596,Claims of global warming have been greatly exa...,disagree
4,4,1,0,t12,0.035201,0.959757,0.005042,The Intergovernmental Panel on Climate Change ...,neutral


In [206]:
dallas_sentence_scores['sentence'][5]

'Evidence now leans against global warming resulting from human-related greenhouse gas emissions.'

# Create train / val / test split

In [10]:
all_train = sentence_df[['round','batch','sent_id','disagree','neutral','agree',
                        'max_prob_label','sent_clean']]

print(len(all_train), len(all_train[all_train['max_prob_label']=='agree']), 
      len(all_train[all_train['max_prob_label']=='neutral']),
     len(all_train[all_train['max_prob_label']=='disagree']))

2050 783 863 404


In [11]:
sub_train = all_train[all_train['sent_clean'].str.\
                      contains("climate|change|global|warming")==True]
print(len(sub_train))

1381


In [13]:
300+400+400+500+500

2100

In [14]:
#randomly select tweets that go to validation set or training set
all_train = shuffle(all_train,random_state=123)
all_train = all_train.drop_duplicates('sent_clean')
num_sents = len(all_train)

print(num_sents)
infl_val_agr = all_train[all_train['max_prob_label']=='agree']
infl_val_neut = all_train[all_train['max_prob_label']=='neutral']
infl_val_dis = all_train[all_train['max_prob_label']=='disagree']
print(len(infl_val_agr), len(infl_val_neut), len(infl_val_dis))

train_pct = .9

2042
779 862 401


In [15]:
training_data = all_train[:int(num_sents*train_pct)]

val_data = all_train[int(num_sents*train_pct):]

print("Num total sentences", num_sents,
      "\n Num train sentences", len(training_data), 
      "\n Num validation sentences", len(val_data))

Num total sentences 2042 
 Num train sentences 1837 
 Num validation sentences 205


In [16]:
training_data.to_csv(os.getcwd()+'/data_for_classifier/dedup_training_data.csv', sep=',')
val_data.to_csv(os.getcwd()+'/data_for_classifier/dedup_val_data.csv', sep=',')

In [2]:
import os
os.getcwd()

'/Users/yiweiluo/scientific-debates/cc_stance/MTurk_results'